In [1]:
import os
import sys

# Model/Training related libraries
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Logging
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

# Dataloader libraries
from torch.utils.data import DataLoader, Dataset, SequentialSampler
from torchvision import transforms

# Custom models/loss/dataloaders/utilities
from models import SiameseNet_OntologicalLayer, YamnetSiameseNet_OntologicalLayer
from loss import Ontological_Loss, Ontological_Loss_Unweighted
from dataloaders import AudioSet_Siamese, AudioSet_Siamese_Eval, AudioSet_Strong_Siamese, AudioSet_Strong_Siamese_Eval
from sklearn import metrics
import scipy.io

# GPU
device = torch.device("cuda:0")

In [2]:
# Load data
data_dir = './data/'
sounds_data = np.load(data_dir + 'audioset_strong_train1_data.npy', allow_pickle=True)
class1_index = np.load(data_dir + 'audioset_strong_train1_labels_1.npy', allow_pickle=True)
class2_index = np.load(data_dir + 'audioset_strong_train1_labels_2.npy', allow_pickle=True)
# Dataloader
train_data = AudioSet_Strong_Siamese(sounds_data, class1_index, class2_index, 37, 7)
train_args = dict(shuffle = True, batch_size = 256, num_workers=8, pin_memory=True)
train_loader = DataLoader(train_data, **train_args)

sounds_data = np.load(data_dir + 'audioset_strong_val_data.npy', allow_pickle=True)
class1_index = np.load(data_dir + 'audioset_strong_val_labels_1.npy', allow_pickle=True)
class2_index = np.load(data_dir + 'audioset_strong_val_labels_2.npy', allow_pickle=True)

# val_idx = np.random.choice(len(sounds_data), int(0.2*train_data.data.shape[0]/10))
# sounds_data = sounds_data[val_idx]
# class2_index = class2_index[val_idx]
# val_idx_extended = np.zeros((len(val_idx), 10))
# for i in range(len(val_idx)):
#     val_idx_extended[i] = np.arange(val_idx[i], val_idx[i] + 10)
# val_idx_extended = val_idx_extended.reshape(-1, 1).astype('int')
# class1_index = class1_index[val_idx_extended]    

val_data = AudioSet_Strong_Siamese(sounds_data, class1_index, class2_index, 37, 7)
val_args = dict(shuffle = False, batch_size = 256, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_data, **val_args)

eval_data = AudioSet_Strong_Siamese_Eval(sounds_data, class1_index, class2_index, 37, 7)
eval_args = dict(shuffle = False, batch_size = 256, num_workers=8, pin_memory=True, sampler=SequentialSampler(eval_data))
eval_loader = DataLoader(eval_data, **eval_args)


[23  0 16 14]
[ 0 16]


In [ ]:
print(len(sounds_data))
print(len(class1_index))

class1_index_cat = np.concatenate(class1_index, axis=0)
class1_index_cat = [np.asarray(lbl) for lbl in class1_index_cat]
# print(class1_index_cat)
print(class1_index_cat[10].dtype)

print(train_data.labels1[:40])
print(val_data.labels1[:40])
print(train_data.labels2[:40])
print(val_data.labels2[:40])

print(train_data.logits_1[:10])
# print(eval_data.labels1)

print(len(train_data.logits_1))
print(len(train_data.logits_2))
print(len(train_data.data))

In [ ]:
# Load data
data_dir = './test/audioset_strong/'
sounds_data = np.load(data_dir + 'audioset_strong_bal_train_data.npy', allow_pickle=True)
class1_index = np.load(data_dir + 'audioset_strong_bal_train_labels_1.npy', allow_pickle=True)
# class2_index = np.load(data_dir + 'audioset_strong_bal_train_labels_2.npy', allow_pickle=True)

data = np.concatenate(sounds_data, axis=0).astype('float')
seg_per_clip = 10
length = data.shape[0]
labels1 = np.concatenate(class1_index, axis=0)
labels1 = [np.asarray(lbl) for lbl in labels1]
# labels2 = np.concatenate(class2_index, axis=0)        
# labels2 = [np.asarray(lbl) for lbl in labels2]

num_clips = data.shape[0]
num_subclasses = 42
# num_superclasses = 7

logits_1 = np.zeros((num_clips, num_subclasses)).astype('long')
# logits_2 = np.zeros((num_clips, num_superclasses)).astype('long')

for i in range(num_clips):
    logits_1[i][labels1[i].astype('int')] = 1 
    # logits_2[i][labels2[i].astype('int')] = 1


In [ ]:
print(np.where(np.sum(logits_1, axis=0) != 0))
# print(np.sum(logits_2, axis=0))

class_to_condensed = np.zeros((42,))
idx = np.where(np.sum(logits_1, axis=0) != 0)[0]
for i in range(len(idx)):
    class_to_condensed[idx[i]] = i
    
print(class_to_condensed)

condensed_labels = []
for lbls in labels1:
    condensed_labels.append(class_to_condensed[lbls].astype('int'))
# np.save('./data/audioset_strong_eval_labels_1.npy', np.asanyarray(condensed_labels, dtype=object))

In [ ]:
data_dir = './test/audioset_strong/'
# sounds_data = np.load(data_dir + 'audioset_strong_bal_train_data.npy', allow_pickle=True)
class1_index = np.load(data_dir + 'audioset_strong_eval_labels_1.npy', allow_pickle=True)
labels1 = np.concatenate(class1_index, axis=0)
labels1 = [np.asarray(lbl) for lbl in labels1]
condensed_labels = []
for lbls in labels1:
    condensed_labels.append(class_to_condensed[lbls].astype('int'))
np.save('./data/audioset_strong_eval_labels_1.npy', np.asanyarray(condensed_labels, dtype=object))



In [ ]:
print(train_data.logits_1.shape)
print(np.sum(train_data.logits_1, axis=0))
print(np.sum(eval_data.logits_1, axis=0))
print(np.sum(val_data.logits_1, axis=0))

print(eval_data.logits_1.shape)
print(val_data.logits_1.shape)

print(len(sounds_data))
print(len(class1_index))
print(len(class2_index))



In [6]:
# Ontology Layer 
M = np.asarray([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
                [0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
                [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

# for strongly labeled data
condensed_idx = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41]
M = M[:, condensed_idx]
M = M / np.sum(M, axis=1).reshape(-1, 1)

# Siamese Net Model
in_feature_dim = train_data.__getitem__(0)[0].shape[0]
model = SiameseNet_OntologicalLayer(7, 37, in_feature_dim, M)
model.to(device)

# Define Loss function
lambda1 = 1.75
lambda2 = 1
lambda3 = 0.25

# Weights for BCE Loss 
tot1 = np.sum(train_data.logits_1)
pos_weights_1 = np.sum(train_data.logits_1, axis=0)
neg_weights_1 = train_data.logits_1.shape[0] - pos_weights_1
tot2 = np.sum(train_data.logits_2)
pos_weights_2 = np.sum(train_data.logits_2, axis=0)
neg_weights_2 = train_data.logits_2.shape[0] - pos_weights_2
label_weights_1 = torch.tensor(neg_weights_1/pos_weights_1).to(device)
label_weights_2 = torch.tensor(neg_weights_2/pos_weights_2).to(device)

criterion = Ontological_Loss(lambda1, lambda2, lambda3, label_weights_1, label_weights_2)

# Define Optimizer
learningRate = 2e-4
weightDecay = 1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)



In [7]:
### Train ###
def train_model(train_loader, model, epoch, writer):
    
    training_loss = 0
    training_acc_1 = 0
    training_acc_2 = 0
    
    # Set model in 'Training mode'
    model.train()
    
    # enumerate mini batches
    with tqdm(train_loader, ) as t_epoch:
        for i, (input1, input2, target1_1, target1_2, target2_1, target2_2, pair_type) in enumerate(t_epoch):
            t_epoch.set_description(f"Epoch {epoch}")
            
            # Move to GPU
            input1 = input1.to(device).float()
            target1_1 = target1_1.to(device)
            target1_2 = target1_2.to(device)
            
            # print(target1_1.size())
            
            input2 = input2.to(device).float()
            target2_1 = target2_1.to(device)
            target2_2 = target2_2.to(device)
            
            # print(target1_1)
            # print(target1_2)
            # print(pair_type)
            
            # print(pair_type)
            pair_type = pair_type.to(device)

            model.zero_grad()
            optimizer.zero_grad()
            
            # Model output
            outputs = model.forward(input1, input2) 
            targets = (target1_1, target1_2, target2_1, target2_2)
            
            # Loss/Backprop
            loss = criterion(outputs, targets, pair_type)                           
            loss.backward()
            optimizer.step()     

            training_loss += loss.item()
            t_epoch.set_postfix(loss=loss.item())
            
            torch.cuda.empty_cache()
            del input1, input2
            del target1_1, target1_2, target2_1, target2_2, pair_type
            del loss
    
    training_loss /= len(train_loader)
    writer.add_scalar("Loss/train", training_loss, epoch)  
    
    return training_loss


# Validation
def evaluate_model(val_loader, model, epoch, writer):
        
    val_loss = 0

    # Set model in validation mode
    model.eval()
    
    for i, (input1, input2, target1_1, target1_2, target2_1, target2_2, pair_type) in enumerate(val_loader):
        with torch.no_grad():
            
            # Move to GPU
            input1 = input1.to(device).float()
            target1_1 = target1_1.to(device)
            target1_2 = target1_2.to(device)
            
            input2 = input2.to(device).float()
            target2_1 = target2_1.to(device)
            target2_2 = target2_2.to(device)
            
            pair_type = pair_type.to(device)
            
            # Model Output
            outputs = model.forward(input1, input2) # model output
            targets = (target1_1, target1_2, target2_1, target2_2)
            
            # Val loss
            loss = criterion(outputs, targets, pair_type)            
            val_loss += loss.item()
    
    writer.add_scalar("Loss/val", val_loss / len(val_loader), epoch)  

    return loss.item()

In [8]:
def evaluate_model_stats(data_loader, model, reduction='weighted'):
    
    model.eval()
    device = torch.device("cuda:0")
    
    complete_outputs_1 = []
    complete_targets_1 = []
    
    complete_outputs_2 = []
    complete_targets_2 = []
    
    # Evaluate test set in batches
    for i, (input1, target1_1, target1_2) in enumerate(data_loader):
        with torch.no_grad():
            
            # Move to GPU
            input1 = input1.to(device).float()
            target1_1 = target1_1.to(device)
            target1_2 = target1_2.to(device)
            
            batch_size = int(input1.shape[0]/2)
            
            # Model Output
            _, _, out1_1, out1_2, out2_1, out2_2 = model.forward(input1[0:batch_size], input1[batch_size::]) # model output
            
            sigmoid = torch.nn.Sigmoid()
            out1_1 = sigmoid(out1_1)
            out2_1 = sigmoid(out2_1)

            complete_outputs_1.append(torch.cat((out1_1, out2_1)))
            complete_targets_1.append(target1_1)
            
            complete_outputs_2.append(torch.cat((out1_2, out2_2)))
            complete_targets_2.append(target1_2)
    
    
    # Concat batch results 
    complete_outputs_1 = torch.cat(complete_outputs_1, 0)
    complete_targets_1 = torch.cat(complete_targets_1, 0)
    
    complete_outputs_2 = torch.cat(complete_outputs_2, 0)
    complete_targets_2 = torch.cat(complete_targets_2, 0)
    
    # print(complete_outputs_1)
    # print(complete_targets_1)
    
    num_classes_1 = complete_outputs_1.shape[-1]
    num_classes_2 = complete_outputs_2.shape[-1]
    
    segs_per_clip = 10
    
    # Move to CPU
    complete_targets_1 = complete_targets_1.detach().cpu().numpy()
    complete_outputs_1 = complete_outputs_1.detach().cpu().numpy()
    
    complete_targets_2 = complete_targets_2.detach().cpu().numpy()
    complete_outputs_2 = complete_outputs_2.detach().cpu().numpy()
    
    # Average outputs over entire audio clip
    # output_1_avg = np.zeros((int(complete_outputs_1.shape[0]/segs_per_clip), complete_outputs_1.shape[1]))
    # output_2_avg = np.zeros((int(complete_outputs_2.shape[0]/segs_per_clip), complete_outputs_2.shape[1]))
    # for i in range(int(complete_outputs_1.shape[0]/segs_per_clip)):
    #     output_1_avg[i] = np.mean(complete_outputs_1[segs_per_clip*i : segs_per_clip*(i+1)], axis=0)
    #     output_2_avg[i] = np.mean(complete_outputs_2[segs_per_clip*i : segs_per_clip*(i+1)], axis=0)
    
    # print(output_1_avg)
#     pos_weights_1 = np.sum(complete_targets_1[0::segs_per_clip], axis=0)
#     pos_weights_2 = np.sum(complete_targets_2[0::segs_per_clip], axis=0)
#     tot1 = np.sum(complete_targets_1[0::segs_per_clip])
#     tot2 = np.sum(complete_targets_2[0::segs_per_clip])
    
#     weights_1 = pos_weights_1 / tot1
#     weights_2 = pos_weights_2 / tot2

    output_1_avg = complete_outputs_1
    output_2_avg = complete_outputs_2

    pos_weights_1 = np.sum(complete_targets_1, axis=0)
    pos_weights_2 = np.sum(complete_targets_2, axis=0)
    tot1 = np.sum(complete_targets_1)
    tot2 = np.sum(complete_targets_2)
    
    weights_1 = pos_weights_1 / tot1
    weights_2 = pos_weights_2 / tot2
            
    # Level 1 Average precision, AUC metrics
    average_precision_1 = np.zeros((num_classes_1, ))
    auc_1 = np.zeros((num_classes_1, ))
    for i in range(num_classes_1):
        average_precision_1[i] = metrics.average_precision_score(complete_targets_1[:, i], output_1_avg[:, i])
        auc_1[i] = metrics.roc_auc_score(complete_targets_1[:, i], output_1_avg[:, i], average = None)

    # Level 2 Average precision, AUC metrics
    average_precision_2 = np.zeros((num_classes_2, ))
    auc_2 = np.zeros((num_classes_2, ))
    for i in range(num_classes_2):
        average_precision_2[i] = metrics.average_precision_score(complete_targets_2[:, i], output_2_avg[:, i])
        auc_2[i] = metrics.roc_auc_score(complete_targets_2[:, i], output_2_avg[:, i], average = None)
    
    result_dir = './results/'
    scipy.io.savemat(result_dir + 'eval_metrics_test.mat', {'test_mAP_1': average_precision_1, 'test_mAP_2': average_precision_2, 'test_AUC_1': auc_1, 'test_AUC_2': auc_2})
    
    if(reduction=='average'):
        mAP_1 = np.mean(average_precision_1)
        mauc_1 = np.mean(auc_1)

        mAP_2 = np.mean(average_precision_2)
        mauc_2 = np.mean(auc_2)
        
    elif(reduction=='weighted'):
        mAP_1 = np.sum(weights_1*average_precision_1)
        mauc_1 = np.sum(weights_1*auc_1)
        
        mAP_2 = np.sum(weights_2*average_precision_2)
        mauc_2 = np.sum(weights_2*auc_2)
        
    elif(reduction=='none'):
        return average_precision_1, auc_1, complete_outputs_1, average_precision_2, auc_2, complete_outputs_2
    
    return mAP_1, mauc_1, complete_outputs_1, mAP_2, mauc_2, complete_outputs_2




In [ ]:
print(eval_data.logits_1[0])
print(np.sum(eval_data.logits_1, axis=0))
print(np.sum(eval_data.logits_2, axis=0))
print(eval_data.logits_1.shape)
print(eval_data.logits_2.shape)
print(eval_data.labels1[:50])
print(train_data.labels1[:100])

In [ ]:
print(train_data.__getitem__(0))

In [12]:
# Save dirs
model_num = 'siamese_strong_doubleweighted_lambda1_' + str(lambda1) + '_lambda2_' + str(lambda2) + '_lambda3_' + str(lambda3) + '/'
base_dir = './'
model_dir = base_dir + 'models/' + model_num
runs_dir = base_dir + 'runs/' + model_num

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(runs_dir):
    os.makedirs(runs_dir)

# Tensorboard logging
writer = SummaryWriter(runs_dir)
torch.backends.cudnn.benchmark = True

# Training loop
epochs = 75
for epoch in range(epochs):
    
    # Train + Validation
    training_loss = train_model(train_loader, model, epoch, writer)
    val_loss = evaluate_model(val_loader, model, epoch, writer)
    
    # Val Stats
    val_mAP_1, val_auc_1, complete_outputs_1, val_mAP_2, val_auc_2, complete_outputs_2 = evaluate_model_stats(eval_loader, model, reduction='weighted')

    scheduler.step()
    
    # Print log of accuracy and loss
    print("Epoch: "+str(epoch)+", Training loss: "+str(training_loss)+", Validation loss: "+str(val_loss)+", Validation mAP_1/AUC_1: "+str(val_mAP_1)+"/"+str(val_auc_1)+
             ", Validation mAP_2/AUC_2: "+str(val_mAP_2)+"/"+str(val_auc_2))
    
    writer.add_scalar("mAP_1/val", val_mAP_1, epoch)
    writer.add_scalar("AUC_1/val", val_auc_1, epoch)
    writer.add_scalar("mAP_2/val", val_mAP_2, epoch)
    writer.add_scalar("AUC_2/val", val_auc_2, epoch)
    
    # Save model checkpoint
    model_filename = model_dir + 'epoch' + str(epoch) + '.pt'
    if(epoch == 0):
        torch.save(model, model_filename)
    else:
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': training_loss,}, model_filename)

Epoch 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1952/1952 [06:12<00:00,  5.24it/s, loss=7.54]


ValueError: operands could not be broadcast together with shapes (42,) (37,) 

In [ ]:
# Save dirs
model_num = 'siamese_lambda1_' + str(lambda1) + '_lambda2_' + str(lambda2) + '_lambda3_' + str(lambda3) + '/'
base_dir = './'
model_dir = base_dir + 'models/' + model_num
runs_dir = base_dir + 'runs/' + model_num

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(runs_dir):
    os.makedirs(runs_dir)

# Tensorboard logging
writer = SummaryWriter(runs_dir)
torch.backends.cudnn.benchmark = True

# Load model
model_save_dir = model_dir + 'epoch0.pt'
model = torch.load(model_save_dir) 
model.to(device)

# Load saved weights
weights_dir = model_dir + 'epoch43.pt'
state = torch.load(weights_dir)
model.load_state_dict(state['model_state_dict'])

# Training loop
epochs = 40
for epoch in range(50, 75):
    
    # Train + Validation
    training_loss = train_model(train_loader, model, epoch, writer)
    val_loss = evaluate_model(val_loader, model, epoch, writer)
    
    # Val Stats
    val_mAP_1, val_auc_1, _, val_mAP_2, val_auc_2, _ = evaluate_model_stats(eval_loader, model)

    scheduler.step()
    
    # Print log of accuracy and loss
    print("Epoch: "+str(epoch)+", Training loss: "+str(training_loss)+", Validation loss: "+str(val_loss)+", Validation mAP_1/AUC_1: "+str(val_mAP_1)+"/"+str(val_auc_1)+
             ", Validation mAP_2/AUC_2: "+str(val_mAP_2)+"/"+str(val_auc_2))
    
    writer.add_scalar("mAP_1/val", val_mAP_1, epoch)
    writer.add_scalar("AUC_1/val", val_auc_1, epoch)
    writer.add_scalar("mAP_2/val", val_mAP_2, epoch)
    writer.add_scalar("AUC_2/val", val_auc_2, epoch)
    
    # Save model checkpoint
    model_filename = model_dir + 'epoch' + str(epoch) + '.pt'
    if(epoch == 0):
        torch.save(model, model_filename)
    else:
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': training_loss,}, model_filename)

In [10]:
# Test Data
data_dir = './data/'

sounds_data = np.load(data_dir + 'audioset_test_data.npy', allow_pickle=True)
class1_index = np.load(data_dir + 'audioset_test_labels_1.npy', allow_pickle=True)
class2_index = np.load(data_dir + 'audioset_test_labels_2.npy', allow_pickle=True)

eval_data = AudioSet_Siamese_Eval(sounds_data, class1_index, class2_index, 42, 7)
eval_args = dict(shuffle = False, batch_size = 256, num_workers=8, pin_memory=True)
eval_loader = DataLoader(eval_data, **eval_args)

# Load model
model_save_dir = './models/siamese_strong_doubleweighted_lambda1_1.75_lambda2_1_lambda3_0.25/'
model_num = 'epoch0.pt'
model = torch.load(model_save_dir + model_num) 
model.to(device)

# Load saved weights
weights_dir = 'epoch15.pt'
state = torch.load(model_save_dir + weights_dir)
model.load_state_dict(state['model_state_dict'])
model.eval()

test_mAP_1, test_AUC_1, outputs_1, test_mAP_2, test_AUC_2, outputs_2 = evaluate_model_stats(eval_loader, model, reduction='none')

print(test_mAP_1)
print(test_AUC_1)
print(test_mAP_2)
print(test_AUC_2)

# Save results to txt file
result_dir = './results/'
np.savetxt(result_dir + 'test_mAP_1.txt', test_mAP_1)
np.savetxt(result_dir + 'test_AUC_1.txt', test_AUC_1)
np.savetxt(result_dir + 'test_mAP_2.txt', test_mAP_2)
np.savetxt(result_dir + 'test_AUC_2.txt', test_AUC_2)
scipy.io.savemat(result_dir + 'siamese_doubleweighted_lambda1_2_lambda2_1_lambda3_0.25.mat', {'test_mAP_1': test_mAP_1, 'test_mAP_2': test_mAP_2, 'test_AUC_1': test_AUC_1, 'test_AUC_2': test_AUC_2})

import csv
with open('./results/predictions_1.csv', 'w') as f:
    writer = csv.writer(f)
    
    # Class header
    writer.writerow(np.arange(outputs_1.shape[1]))
    
    for i in range(outputs_1.shape[0]):
        writer.writerow(outputs_1[i])
        
        
with open('./results/predictions_2.csv', 'w') as f:
    writer = csv.writer(f)
    
    # Class header
    writer.writerow(np.arange(outputs_2.shape[1]))
    
    for i in range(outputs_2.shape[0]):
        writer.writerow(outputs_2[i])


[0.06311606 0.2349789  0.28584845 0.21351467 0.07582021 0.11721418
 0.24063135 0.3187921  0.26047259 0.00846959 0.07972048 0.03822025
 0.03900044 0.46335556 0.29915556 0.02327851 0.54584518 0.21613161
 0.11977328 0.05249026 0.20177751 0.25433079 0.01739497 0.01436637
 0.00549193 0.11225592 0.03039365 0.05764055 0.00943022 0.01850149
 0.00383499 0.02142827 0.0034585  0.01113409 0.0029845  0.01040271
 0.10495693]
[0.4849965  0.7548158  0.80625304 0.87640509 0.75836296 0.7476617
 0.74867629 0.90321502 0.79816792 0.4686218  0.69421203 0.68850315
 0.73699597 0.86870828 0.88612716 0.70617116 0.71748438 0.79166325
 0.7385268  0.66744381 0.76024377 0.8075298  0.42104    0.21902503
 0.34636247 0.35514999 0.47696628 0.50763718 0.36769679 0.30530701
 0.59708975 0.59931251 0.61342942 0.36548198 0.51235383 0.18438517
 0.5743179 ]
[0.31704265 0.10949412 0.39397953 0.65275689 0.12372001 0.49159679
 0.16241909]
[0.72181951 0.53032441 0.59207832 0.81420354 0.7501865  0.66301432
 0.57262068]


In [11]:
print(eval_data.data.shape)
print(eval_data.logits_1.shape)
print(eval_data.logits_2.shape)
print(eval_data.labels1)

print(np.mean(test_mAP_1))
print(np.mean(test_mAP_2))
print(np.mean(test_AUC_1))
print(np.mean(test_AUC_2))

(203710, 128)
(20371, 42)
(20371, 7)
[array([5]) array([ 1, 19]) array([16]) ... array([37]) array([25])
 array([ 1, 26, 36])]
0.1236652059062863
0.32157272611331
0.6177389457996932
0.6634638981250486


In [ ]:
import csv

with open('./results/targets_1.csv', 'w') as f:
    writer = csv.writer(f)
    
    writer.writerow(np.arange(42))
    
    
    for i in range(len(eval_data.labels1)):
        targets_1 = np.zeros((42, ))
        targets_1[eval_data.labels1[i]] = 1
        writer.writerow(targets_1)    
    
with open('./results/targets_2.csv', 'w') as f:
    writer = csv.writer(f)
    
    writer.writerow(np.arange(7))
    
    
    for i in range(len(eval_data.labels2)):
        targets_2 = np.zeros((7, ))
        targets_2[eval_data.labels2[i]] = 1
        writer.writerow(targets_2)   
    

In [ ]:
print(len(eval_data.data))
print(outputs_1.shape)
print(len(eval_data.labels1))

In [ ]:
class9_idx = np.where(train_data.logits_1[:, 9] == 1)[0]
print(train_data.labels1[class9_idx])